In [ ]:
!pip install chainladder pycaret

# WS-3 - Introduction to Machine Learning
**CLRS 2023**<br>
Monday, September 11, 2023 8:00 AM – 11:30 AM East Coast USA Time<br>
Location: Hamilton/Indian River<br>
CE: 3.6<br>
Non-Recorded Session

Workshop Speaker(s)
* John Bogaardt, FCAS, MAAA - Chief Actuary WCF Insurance
* Lahiru Somaratne, MSc in Actuarial Science

This 3.5 hour workshop will provide an introduction to machine learning for Actuaries in Python. The workshop is hands-on and will explore the fitting and tuning of several traditional machine learning models using the scikit-learn API. We will briefly expand on the scikit-learn workflow as a modeling framework for reserving through the chainladder-python library. Finally, we will kick it up a notch and explore deep learning examples with Tensorflow/Keras. Emphasis will be on tools and workflow more than a deep dive in theory. The workshop will only briefly cover Python basics, so participants with beginner Python knowledge or fluency in another programming language would have an advantage. We will be using Google Colab for hands-on instruction and participants will need a google account to access Colab. Participants may choose to set up their own environment but no support will be given by the instructors

## Basic Python Syntax - Crash Course

In [ ]:
1 + 1

In [ ]:
x = 5
y = 2
z = x**y
z

In [ ]:
if x > y:
    print('x is greater than y')
elif x == y:
    print('x is equal to y')
else:
    print('x is less than y')

A list is what you would expect.

In [ ]:
my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
my_list

But lists can be sliced...

In [ ]:
my_list[1:3]

In [ ]:
my_list[1:]

In [ ]:
my_list[-3:]

In [ ]:
my_list[::-1]

We can loop over list elements.

In [ ]:
accumulate = 0
for item in my_list:
    accumulate = accumulate + item
print(accumulate)

A "comprehension" is a for loop and a list put together in one line.

In [ ]:
[2*item for item in my_list]

Dictionaries are like lists with labels. They are key/value pairs.

In [ ]:
my_dict = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5}
my_dict

You can access any `value` of a dictionary by its `key`.

In [ ]:
my_dict['b']

However dictionaries make no guarantees about their order, so slicing is not permitted.

In [ ]:
my_dict[:3]

But you can loop over all key/value pairs in a dictionary with a "comprehension".

In [ ]:
{key: value**2 for key, value in my_dict.items()}

A function is a stateless piece of code. Given some inputs it creates an output. These are useful for encapsulating logic into a shortened name. Functions are like verbs. They do things.

In [ ]:
def exponent(x=1, y=0):
    return x**y

exponent(5, 2)

A class is a blueprint for a stateful object. An object can have properties (attributes), and they can also do things (methods). Methods are just a fancy name for functions that are tied to a class.

In [ ]:
class Exponent:
    def __init__(self, x=1, y=0):
        self.x = x
        self.y = y

    def exponent(self):
        return self.x ** self.y

With a class defined, we can create instances or objects of that class.

In [ ]:
instance = Exponent(5, 2)

...and get access to all the pre-defined attributes and methods.

In [ ]:
print(instance.x, instance.y)
instance.exponent()

**Everything is an object in python!**

## The Scientific Python Stack
<img src="https://jupytearth.org/_images/python-stack.png" width="1000"/>

## Importing Packages

The python standard library is useful for basic scripting of computer tasks, but it is not that great for data and analysis.

All of these packages have their own **functions** and **classes** stored in one or more **modules**.

In [ ]:
import matplotlib.pyplot as plt
import chainladder as cl
import numpy as np
import pandas as pd

plt.style.use('ggplot')
%config InlineBackend.figure_format = 'retina'

# The Basics of Machine Learning with Scikit-Learn
`sklearn` is the defacto standard Machine Learning  API in python.

How do you get the most out of your data with Machine Learning with `sklearn`? An opinionated view:

| Technique | Return on Investment |
|-|-|
|Feature Engineering|⭐⭐⭐⭐⭐|
|Model selection|⭐⭐⭐⭐|
|Hyperparameter tuning|⭐⭐⭐|
|Model Ensembling|⭐⭐⭐|
|Feature Preprocessing|⭐⭐|


Let's explore the API.

## Estimators

Almost everything in `sklearn` is an estimator.  Estimators have properties that are worth remembering.

Estimators are python objects that carry out a Machine Learning task. These objects can do things (`fit`, `predict`) and have properties associated with them.

Depending on the algorithm you wish to use, you will need to import the specific `sklearn` class from which you'll create your estimator. This will typically follow the following structure:

```python
from some_module import Estimator
estimator = Estimator()
```
Estimators are instances of a class (which is denoted by Uppercase naming convention). Creating an instance of a model is as simple as adding parenthesis at the end of the class name.

Let's import the LinearRegression class and create an instance.

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model

👉 _All estimator objects can optionally be configured with **hyper**parameters to uniquely specify the model being built. This is done ahead of exposing any data to the model._

Hyperparameters are parameters that are set before the learning process of a machine learning algorithm. Unlike the model's parameters, which are learned from the data during training, hyperparameters are predefined and determined by the user or data scientist. These hyperparameters influence the behavior and performance of the learning algorithm.

```python
estimator = Estimator(param1=1, param2=2)
```

`sklearn` has sensible default hyperparameters. These can be viewed and optionally overridden.

In [ ]:
model.get_params()

Note that we can set our **hyperparameters** in the absence of looking at any data. These parameters are there to tell the model how it should behave during the fitting process.

In [ ]:
model = LinearRegression(fit_intercept=False)
model

👉 _All estimator objects expose a `fit` method that takes as input, `X`_:

```python
estimator.fit(X, y)
```
Models also optionally support a response, `y` or and weights, `sample_weight`.


## Synthetic data
Let's create some _independent_ variables, **X** to explore the concept.

In [ ]:
group = np.random.rand(200)
group = np.where(group<.4, 'a', np.where(group<.8, 'b', 'c'))
age = np.linspace(16, 100, 200)
X = pd.DataFrame({'group': group, 'age': age})
X.head()

Let's also syntehsize a _dependent_ variable, **y** that depends on the values of **X**.

In [ ]:
y = 30 + X['group'].map({'a': 1.5, 'b': 1.0, 'c': 0.5}) * X['age'] + np.random.normal(0, 15, 200)
y[:5]

We know the linear equation that generated this data, but because we added an error term, we can see the _noise_ in our data.

In [ ]:
pd.concat((X, pd.Series(y, name='y')), axis=1).plot(
    kind='scatter',
    x='age', y='y',
    title='Simple Regression Data',
    c=X['group'].map({'a': 'blue', 'b': 'green', 'c': 'orange'}));

## The `fit` method
Let's fit our model to our data just using `age`.

In [ ]:
model = LinearRegression().fit(X[['age']], y)
model

It looks like nothing happened, but in actuality, our model has be fitted to the data.  The `model` object itself displays the same, but it now has additional features.  

👉 _When a model is fit, the fitted attributes are denoted by a trailing **underscore**._

```python
estimator.estimated_param_
```

The use of the underscore is a key API design style of scikit-learn that allows for the quicker recognition of fitted parameters vs hyperparameters:

In [ ]:
f'The slope of the model is {model.coef_} and the intercept is {model.intercept_}.'

## The `score` method
Can we do statistical inference with `sklearn`?

Only in limited ways at an overall level. There are no coefficient-specific t-statistics, p-values, etc, associated with this model retained by `sklearn`. The machine learning focus of the library promotes overall model performance over and will have metrics for model accuracy, R-squared, AUC, etc.  

👉 _Scoring can only be invoked after a model has been `fit`._

By default regression problems use _R2_ as a scoring metric and classification uses _accuracy_. These can be changed and alternatives are found in the `sklearn.metrics` module.

In [ ]:
print('Model R2:',  model.score(X[['age']], y))

## The `predict` method

We can also make predictions with a fitted model using the `predict` method.

In [ ]:
predictions = model.predict(X[['age']])
predictions[:10]

In [ ]:
ax = pd.concat((X, pd.Series(y, name='y')), axis=1).plot(
    kind='scatter', x='age', y='y',
    title='Linear Approximation to Data');

pd.concat(
    (X[['age']], pd.Series(predictions, name='y')),
    axis=1
).plot(
    kind='line', x='age', y='y',
    ax=ax, legend=False,
    linestyle='dashed');

Fitting the entire independent variableset including our `group` feature doesn't work. Why?

In [ ]:
model.fit(X, y)

## Transformers (Preprocessing)

`sklearn` comes with a a toolbox of preprocessing **Transformers** that can help specific algorithms react more appropriately to your data. Many algorithms have limitations. For example they may:
 * Can only deal with numeric values (OneHotEncoder or OrdinalEncoder)
 * Hate missing values (SimpleImputer or KNNImputer)
 * Use Euclidean distance for decision boundaries (StandardScaler)
 * Don't deal well with non-linear data (KBinsDiscretizer)
 * Don't explicitely recognize interactions (PolynomialFeatures)


A **Transformer** is just like the Estimators we learned about above with a couple of exceptions:
1. They transform our feature-space **X**
2. They have a `transform` method instead of a `predict` and `score` method.

We can see that the `linear_model` doesn't like categorical data.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
ohe.fit(X[['group']])

Once fit, transformers have fitted attributes (denoted by trailing underscore).

In [ ]:
ohe.categories_

We can then use our transform our featureset.

In [ ]:
print(X['group'].head())
ohe.transform(X[['group']]).todense()[:5]

In [ ]:
X_transformed = pd.concat((
    pd.DataFrame(
        ohe.transform(X[['group']]).todense(),
        columns=ohe.categories_[0]),
    X['age']),
    axis=1)

X_transformed.head()

We can fit our model to this featureset, and see we get better performance.

In [ ]:
model.fit(X_transformed, y)
model.score(X_transformed, y)

Still, the form is not correct. This assumes each group and age are independent features. However, the source equation is an interaction between age and group.

In [ ]:
pd.Series(model.coef_, index=model.feature_names_in_)

Let's also create the interaction terms we know are in the model (because we synthesized this data).

In [ ]:
X.head()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
pf.fit(X_transformed)

In [ ]:
X_transformed = pd.DataFrame(
    pf.transform(X_transformed),
    columns=pf.get_feature_names_out())
X_transformed.head()

We can now fit our model assuming the age:group interactions are the only relevant features.

In [ ]:
model = LinearRegression().fit(X_transformed[['a age', 'b age', 'c age']], y)

In [ ]:
print('Compared to true coefficients of [1.5, 1.0, 0.5], our model coefficients are', model.coef_)
print('Compared to true intercept of 30, our model coefficients are', model.intercept_)
print('Model R2:', model.score(X_transformed[['a age', 'b age', 'c age']], y))

😩 These transformers are tedious. When new data comes in, the same transformations have to be applied to make a prediction.  

## Pipelines (Composite Estimators)

Piecing transformers and predictors together is a very common pattern and it is nice to do so succintly. With `Pipeline`s, one can create custom end-to-end models.

We do so by specifying a list of steps that should be executed in order.


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

pipe = Pipeline([
    ('group_transform', ColumnTransformer([('ohe', OneHotEncoder(), ['group']), ('pass', 'passthrough', ['age'])])),
    ('create_interaction', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
    ('retain_only_interactions', ColumnTransformer([('keep', 'passthrough', [6, 8, 9])])),
    ('model', LinearRegression())
])
pipe

Setting up a `Pipeline` can be complex, but once done, it makes everything else easier, like fitting:

In [ ]:
X.head()

In [ ]:
pipe.fit(X, y)

scoring...

In [ ]:
pipe.score(X, y)

or making new predictions on raw data...

In [ ]:
print(X.iloc[10:11])

print('prediction', pipe.predict(X.iloc[10:11]))

Most importantly, `Pipeline`s encapsulate data transformations.

👉 _`Pipeline`s substantially minimize the risk of data leakage and are crucial for managing model generalization when preprocessing data is involved._


Let's look at the Regression Pipeline we created.

In [ ]:
ax = pd.concat((X, pd.Series(y, name='y')), axis=1).plot(
    kind='scatter',
    x='age', y='y',
    title='Simple Regression Data',
    c=X['group'].map({'a': 'blue', 'b': 'green', 'c': 'orange'}));

pd.DataFrame({'age': X[X['group']=='a']['age'], 'y': pipe.predict(X[X['group']=='a'])}).plot(
    kind='line', x='age', y='y', ax=ax, legend=False, linestyle='dashed', color='blue');
pd.DataFrame({'age': X[X['group']=='b']['age'], 'y': pipe.predict(X[X['group']=='b'])}).plot(
    kind='line', x='age', y='y', ax=ax, legend=False, linestyle='dashed', color='green');
pd.DataFrame({'age': X[X['group']=='c']['age'], 'y': pipe.predict(X[X['group']=='c'])}).plot(
    kind='line', x='age', y='y', ax=ax, legend=False, linestyle='dashed', color='orange');

# A Classification Example

These are the very basics of `sklearn`, but let's try something on little more interesting with reserving triangles.

`sklearn` has estimators for clustering, classification, and regression.



<img src="https://scikit-learn.org/stable/_static/ml_map.png" width="1000"/>


Diving deep into all areas is beyond the scope of this seminar. We will instead explore the scope of ML through a classification use-case.

**Can we determine the line of business of a loss triangle from the triangle itself?**


## The Dataset
### A `chainladder` Triangle

Let's explore the `clrd`  ([CAS Loss Reserve Database](https://www.casact.org/publications-research/research/research-resources/loss-reserving-data-pulled-naic-schedule-p)) dataset stored within the `chainladder` python package.

In [ ]:
clrd = cl.load_sample('clrd')
clrd

We are using a `chainladder.Triangle` to explore the dataset.

In [ ]:
type(clrd)

The `Triangle` object is like a pandas DataFrame with an individual triangle as a datatype.  We can have different rows of a triangle representing Companies and Lines of Business.  We can have different rows covering the different measures like Paid Loss, Premium, etc.  The result is a 4D object (Table of Triangles):


<img src="https://raw.githubusercontent.com/casact/chainladder-python/master/docs/images/triangle_graphic.PNG" width="1000" />

Our `clrd` triangle expressed as a DataFrame abstraction:

In [ ]:
pd.DataFrame(
    index=clrd.index,
    columns=list(clrd.columns),
    data=np.repeat(np.repeat([[str(clrd.shape[2:])]], clrd.shape[0], axis=0), clrd.shape[1], axis=1)).head(10)

### Similarity to Pandas
The `Triangle` object is very similar to a pandas DataFrame and uses a subset of the pandas API for data manipulation. There are several advantages in having a dedicated Triangle object:

* Actuaries work with sets of triangles. DataFrames, being two dimensional, support single triangles with grace but become unwieldy with multiple triangles.
* We can carry through the meaningful pandas functionality while also supporting triangle specific methods not found in pandas
* Improved memory footprint with sparse array representation in backend
* Calculated fields with “virtual” columns allows for lazy column evaluation of Triangles as well as improved memory footprint for larger triangles.

Ultimately, there are a lot of things pandas can do that are not relevant to reserving, and there are a lot of things a Triangle needs to do that are not handled easily with pandas.  That said, a lot of pandas functionality works with Triangles.  For example, we can slice individual cells out of our table of Triangles by expressing a row and column combination. When looking at a single cell, we get the detailed representatin of that cell:

In [ ]:
clrd.loc[('Agway Ins Co', 'comauto')]['CumPaidLoss']

We can calculate aggregate values over all "rows" of the `Triangle`:

In [ ]:
clrd['CumPaidLoss'].sum()

We can filter the triangle to a specific LOB.

In [ ]:
clrd[clrd['LOB']=='ppauto']['CumPaidLoss'].sum().T.plot(xlabel='Age', title='Personal Auto Paid Loss');

We can also do things that pandas doesn't support such as explore the age-to-age factors or `link_ratio` of any triangle.

In [ ]:
clrd[clrd['LOB']=='ppauto']['CumPaidLoss'].sum().link_ratio.heatmap()

Creating a new column in a Triangle looks identical to the same operation in pandas.

In [ ]:
clrd['CaseIncurLoss'] = clrd['IncurLoss'] - clrd['BulkLoss']
clrd

### Feature Engineering

Feature engineering is the process of selecting, transforming, and creating new features from raw data to improve the performance of machine learning models. It is a crucial step in a machine learning as it can have a significant impact on the accuracy and effectiveness of the models.

The importance of feature engineering cannot be understated.  In deep learning frameworks, automatic feature engineering is one of the key reasons why it has become so powerful and successful in various machine learning tasks. Traditional machine learning algorithms often rely on handcrafted features, where domain experts manually engineer relevant features from raw data. However, this process can be time-consuming, error-prone, and may not capture all the underlying patterns in complex datasets.

`sklearn` is a traditional machine learning framework, so we are going to rely on our reserving expertise (and `chainladder`) to engineer features. As actuaries, we know that some lines of business are short-tailed and some are long-tailed. This information is encoded in the loss development patterns of a triangle. It makes sense that these patterns might also prove useful for a machine learning algorithm trying to predict line of business.

#### Link Ratios
`chainladder` has estimators that follow the sklearn convention too. We can define a model and fit it to some data.

We can define a model by creating an instance of a `chainladder` transformer. Here we take a simple average of link-ratios for each development age of a Triangle.

Here `cl.Development` is a `sklearn`-compliant estimator and `average` is a _hyperparameter_.

In [ ]:
dev_model = cl.Development(average='simple')
dev_model

Reserving is more like an unsupervised technique. We don't typically have a response variable, `y`.  Ignoring `y` is expected in `chainladder`, and for `sklearn` unsupervised techniques.

In [ ]:
dev_model.fit(X=clrd['CumPaidLoss'])

Like sklearn, fitting our estimator exposes fitted parameters. The `Development` estimator has a `ldf_` parameter representing the selected age-to-age factors which are also represented by the `Triangle` object.

In [ ]:
paid_link_ratios = dev_model.ldf_
paid_cdfs = dev_model.cdf_

paid_link_ratios

Like pandas we can slice into our Triangle to see a subset of the data.

In [ ]:
paid_link_ratios.loc['Allstate Ins Co Grp'].loc['wkcomp']

Let's derive the `ldf_` patterns for `CaseIncurLoss`:

In [ ]:
inc_link_ratios = dev_model.fit(X=clrd['CaseIncurLoss']).ldf_
inc_cdfs = dev_model.cdf_

In [ ]:
inc_link_ratios.loc['Allstate Ins Co Grp'].loc['wkcomp']

#### Paid to Incurred Ratio
The ratio of paid amounts to incurred amounts gives information on how fast claims are reported vs settled. A line like Workers' Compensation has fast reporting but slow settlement. Medical Malpractice might have slow reporting and slow settlement.

In [ ]:
clrd['Paid-to-Incurred'] = clrd['CumPaidLoss'] / clrd['CaseIncurLoss']
paid_to_inc = clrd['Paid-to-Incurred'].mean(axis='origin')

In [ ]:
paid_to_inc.loc['Allstate Ins Co Grp'].loc['wkcomp']

#### Incremental Paid on Prior Case
Another variation on the paid to incurred is the incremental paid on prior case reserve. It measures what percentage of the prior year-end case reserves are paid out in the subsequent year.

In [ ]:
clrd['CaseReserves'] = clrd['CaseIncurLoss'] - clrd['CumPaidLoss']

increm_paid_per_case = (
    clrd['CumPaidLoss'].cum_to_incr().iloc[..., 1:] /
    clrd['CaseReserves'].iloc[..., :-1].values
).mean('origin')

In [ ]:
increm_paid_per_case.loc['Allstate Ins Co Grp'].loc['wkcomp']

### Tidy Data

In order to explore this possibility, we need to **tidy** our dataset up.

1. Each variable in the dataset should be represented by its own column.
2. Each row in the dataset represents a separate observation or case. The dataset should contain individual records or instances, where each row corresponds to a unique observation.
3. Each cell in the dataset should hold a single value. This means that there should be no cells that contain multiple values or combine different types of information.

Our data now has one row per observation (triangle). There are 725 observations with 55 features where ach feature represents a origin/development combination from a 10 x 10 triangle.

In [ ]:
paid_link_ratios = paid_link_ratios.to_frame()
paid_link_ratios.columns = [c + ' Paid Link Ratio' for c in paid_link_ratios.columns]

inc_link_ratios = inc_link_ratios.to_frame()
inc_link_ratios.columns = [c + ' Incurred Link Ratio' for c in inc_link_ratios.columns]

paid_to_inc = paid_to_inc.to_frame()
paid_to_inc.columns = [str(c) + ' Paid to Incurred' for c in paid_to_inc.columns]

increm_paid_per_case = increm_paid_per_case.to_frame()
increm_paid_per_case.columns = [str(c) + ' Incremental Paid to Prior Case' for c in increm_paid_per_case.columns]

paid_cdfs = paid_cdfs.to_frame()
paid_cdfs.columns = [c + ' Paid CDF' for c in paid_cdfs.columns]

inc_cdfs = inc_cdfs.to_frame()
inc_cdfs.columns = [c + ' Incurred CDF' for c in inc_cdfs.columns]

data = pd.concat([paid_link_ratios, inc_link_ratios, paid_to_inc, increm_paid_per_case, paid_cdfs, inc_cdfs], axis=1)
data.head()

### Removing sparse outliers
The `clrd` dataset has a lot of very sparse triangles that don't have any meaningful information in them.

In [ ]:
clrd.loc['Adriatic Ins Co', 'CumPaidLoss'].loc['othliab']

It is usually a good idea to eliminate noisy or non-informative records from our training data. Bad records typically do not improve model performance and may actually hurt it.

Let's define how "full" each triangle is by counting the number of Premium entries each has.

In [ ]:
prem_density = np.minimum(clrd['EarnedPremDIR'], 1).sum('origin').sum('development').to_frame() / 55

In [ ]:
prem_density.loc['Adriatic Ins Co'].loc['othliab']

...and optionally remove lightly populated triangles.

In [ ]:
data = data.loc[prem_density[prem_density>.5].index]

print(sum(prem_density<=0.5), 'records have been removed for being sparse.', sum(prem_density>0.5), 'have been retained.')

## Response encoding
As was the case with our featureset, our response variable needs to be numeric for it to work with several ML algorithms.

In [ ]:
data.head()

We want to predict `LOB`,which happens to be a categorical value.

In [ ]:
data.index.get_level_values('LOB')

More scikit-learn transformers to the rescure.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(data.index.get_level_values('LOB'))

In [ ]:
y = le.transform(data.index.get_level_values('LOB'))
print(le.classes_)
y[:10]

In [ ]:
X = data.reset_index(drop=True)
X.head()

## Data Splitting

The train-test split is a crucial step in the machine learning process, and it serves the improtant purpose of testing whether our models generalize well. Overfitting occurs when a model performs well on the training data but fails to generalize to new data. By evaluating the model on a separate test set, you can assess whether it has learned to capture meaningful patterns or if it is merely memorizing the training data. This helps in detecting and mitigating overfitting issues.

There are several splitting strategies available, but we will use a simple random splitter with 75% in training and 25% in test.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size = 0.25, random_state = 42)

## Our First Model

Let's start with the `KNeighborsClassifier`. It works based on the idea that similar data points tend to have similar labels or target values.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
print(f'Model Training Accuracy: {knn_model.score(X_train, y_train)}')

Let's also try a `RandomForestClassifier` which is an ensemble method that combines multiple decision trees to make a prediction.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
print(f'Model Training Accuracy: {rf_model.score(X_train, y_train)}')

In classification problems, looking beyond accuracy at precision, recall, and F1 score is informative, especially for multinomial classification. These are difficult to remember terms, but are nicely encapsualted in the `confusion_matrix` which is a matrix of actual classification vs predicted classification.

In [ ]:
from sklearn.metrics import confusion_matrix

cm = pd.DataFrame(
    confusion_matrix(y_train, rf_model.predict(X_train)),
    index=le.classes_,
    columns=le.classes_)
cm.style.background_gradient(cmap='Blues', axis=1)

Accuracy is just sum of the diagonal of the confusion matrix over all entries in the confusion matrix.

In [ ]:
print(f'Model Training Accuracy: {np.diag(cm).sum() / cm.sum().sum()}')

This is amazing accuracy! But we're skeptical that it is representative of how good the model will perform on our test dataset.  That said, we don't want to look at our test dataset because if we use performance on test to refine our model, then we are leaking information from our test set back into training.

👉 _Never look at your holdout dataset until you are done with all training iterations of your models._

We overcome this issue with cross-validation on our training dataset.

## Cross Validation
We can use cross-validation to emulate the results expected on a true holdout set.
![](https://upload.wikimedia.org/wikipedia/commons/4/4b/KfoldCV.gif)

In `sklearn`, this is achieved by passing the estimator and the data into the `cross_val_score` function. `cross_val_score` defaults to 5 80/20 random splits of our training data. It trains on the 80% and tests on the 20%.

In [ ]:
from sklearn.model_selection import KFold, cross_validate

cv=KFold(n_splits=5, shuffle=True, random_state=42)
cv

In [ ]:
rf_cv_score = cross_validate(rf_model, X_train, y_train, cv=cv)
print(rf_cv_score['test_score'])
print('Average Cross-Validated Accuracy:', rf_cv_score['test_score'].mean())

This is woefully short of the accuracy on the training data. Is our model providing any value at all?  It's certainly better than KNeighbors.

In [ ]:
knn_cv_score = cross_validate(KNeighborsClassifier(), X_train, y_train, cv=cv)
print(f'KNeighbors Cross-Validated Accuracy:', knn_cv_score['test_score'].mean())

## The NULL Model

We can randomly guess a classification and (assuming data is evenly distributed) expect 1/6 or 17% accuracy from chance. However, our data is not evenly distributed.

In [ ]:
pd.Series(
    le.inverse_transform(y_train)
).value_counts().plot(
    kind='bar',
    title='Frequency of LOBs in Training Dataset');

We can use `sklearn`s Dummy estimators to find out. Like all other estimators, it conforms to the same API and can be used with cross-validation, pipelines, etc.

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_model_cv = cross_validate(DummyClassifier(), X_train, y_train, cv=cv)

print("Dummy Accuracy:", dummy_model_cv['test_score'].mean())

Our RandomForestClassifier is better than the DummyClassifer, so its providing at least some value, but is it the best it can be?

## Parameter Tuning
`sklearn` has sensible defaults for hyperparameters, but changing these will alter how the model works and ultimately will have consequences for model performance. `GridSearchCV` is a very simple and brute-force approach to testing various hyperparameters.  It is effectively a loop over all hyperparameters you want to test (as specified by `param_grid`).

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

rf_grid = GridSearchCV(
    RandomForestClassifier(),
    param_grid={
        'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
    },
    cv=cv)

rf_grid.fit(X_train, y_train)

In [ ]:
pd.DataFrame(rf_grid.cv_results_).set_index('param_n_estimators')[['mean_test_score']].plot(
    title='Average Cross-validated accuracy by Forest Size');

When tuning more than one hyperparameter, `GridSearchCV` will fit each unique combination of your parameter grid.  Here we have 1x4x3=12 parameter sets, cross-validated 5 times, yields 60 models.  

In [ ]:
%%time
rf_grid = GridSearchCV(
    RandomForestClassifier(),
    param_grid={
        'class_weight': ['balanced'],
        'n_estimators': [250, 500, 750, 1000],
        'max_depth': [10, 20, 30],}, cv=cv)

rf_grid.fit(X_train, y_train)

In [ ]:
pd.pivot_table(
    data=pd.DataFrame(rf_grid.cv_results_),
    index=['param_n_estimators'],
    columns='param_max_depth',
    values='mean_test_score'
).style.background_gradient(cmap='Blues', axis=None)

In [ ]:
tuned_rf_cv_score = cross_validate(rf_grid.best_estimator_, X_train, y_train, cv=cv)
print(f'Our Tuned Model accuracy:', tuned_rf_cv_score['test_score'].mean())

`GridSeachCV` is horribly inefficient. That said, the approach of testing various hyperparameters is fundamentally how all hyperparameter optimization routines work. Other routines will only run combinations that are likely to produce model improvement.  For further reference, see `sklearn` documentation of [Hyperparameter Optimizers](https://scikit-learn.org/stable/modules/classes.html#hyper-parameter-optimizers)
or third-party packages such as [Optuna](https://optuna.org/) or [Hyperopt](http://hyperopt.github.io/hyperopt/) for more scalable approaches.

## Gradient Boosted Machines (GBM)

In recent years, gradient boosted machine, specifically libraries like LightGBM and XGBoost, have become increasingly popular in the field of machine learning and data science. They tend to be more robust to many ML painpoint such as:
1. imbalanced data (classification)
2. computational efficiency
3. missing data
4. outliers
5. Non-linear relationships
6. accuracy

Much of the `Pipeline` preprocessing work we did in "The Basics" with Linear Models become much less important with GBM. As modeling competitions go for tabular data, GBM dominates the rankings.


These libraries are also completely compatible with the `sklearn` API.

In [ ]:
from lightgbm import LGBMClassifier

gbm_cv_score = cross_validate(LGBMClassifier(verbose=-1), X_train, y_train, cv=cv)
print(f'Our Tuned Model accuracy:', gbm_cv_score['test_score'].mean())

Like `sklearn`-native estimators, they can be tuned as well.

In [ ]:
%%time
gbm_grid = GridSearchCV(
    LGBMClassifier(verbose=-1),
    param_grid={
        'num_leaves': [2, 5, 10, 15],
        'learning_rate': [.01, .1],
        'reg_alpha': [.5, 1],
        'reg_lambda': [0, .5,]})

gbm_grid.fit(X_train, y_train)

In [ ]:
pd.pivot_table(
    pd.DataFrame(gbm_grid.cv_results_),
    index=['param_num_leaves', 'param_learning_rate'],
    columns=['param_reg_alpha', 'param_reg_lambda'],
    values='mean_test_score', aggfunc='mean'
).style.background_gradient(cmap='Blues', axis=None)

In [ ]:
tuned_gbm_cv_score = cross_validate(gbm_grid.best_estimator_, X_train, y_train, cv=cv)
print('LightGBM Tuned Model accuracy:', tuned_gbm_cv_score['test_score'].mean())

## Ensembling

Another strategy to improve ML performance is to blend multiple algorithms together. Different algorithms have differing strengths and weaknesses.  Combining the effects of different models can help

In [ ]:
from sklearn.ensemble import StackingClassifier

sc = StackingClassifier(
    estimators=[
        ('knn', KNeighborsClassifier()),
        ('rf', RandomForestClassifier()),
        ('gbm', LGBMClassifier(verbose=-1))])

sc.fit(X_train, y_train)

In [ ]:
stacked_cv_score = cross_validate(sc, X_train, y_train, cv=cv)
stacked_cv_score['test_score'].mean()

As with all other Estimators, Ensembles also support hyperparameter optimization. In order to avoid hyperparameter name clashes between different sub-estimators,

_👉 `StackingClassifier` and `Pipeline` use a **double underscore** convention to separate the sub-estimator name and hyperparameter name._

In this way, we are not optimizing any one model, but the entire ensemble.

In [ ]:
%%time
stack_grid = GridSearchCV(
    sc,
    param_grid={
        'knn__n_neighbors': [3, 8],
        'rf__max_depth': [10, 30],
        'gbm__num_leaves': [5, 15]})

stack_grid.fit(X_train, y_train)

In [ ]:
tuned_stacked_cv_score = cross_validate(stack_grid.best_estimator_, X_train, y_train, cv=cv)
tuned_stacked_cv_score['test_score'].mean()

## Model Generalization

At this point, we are seeing diminishing returns with our experiments. We've used `cross_val_score` as a proxy for whether our model will generalize well.  On that basis, the tuned LightGBM model provides the greatest accuracy.


In [ ]:
pd.Series({
    'Dummy': dummy_model_cv['test_score'].mean(),
    'Default KNeighbors': knn_cv_score['test_score'].mean(),
    'Default Random Forest': rf_cv_score['test_score'].mean(),
    'Tuned Random Forest': tuned_rf_cv_score['test_score'].mean(),
    'Default LightGBM': gbm_cv_score['test_score'].mean(),
    'Tuned LightGBM': tuned_gbm_cv_score['test_score'].mean(),
    'Default Stacking': stacked_cv_score['test_score'].mean(),
    'Tuned Stacking': tuned_stacked_cv_score['test_score'].mean()
}).sort_values(ascending=False).plot(kind='barh', title='Cross-validated Accuracy Comparison');

### Performance on Test

We should expect our cross-validated performance to be similar to our holdout performance if:
1. Train/Test Split data distributions are not biased
2. There is no data leakage between Train/Test

In [ ]:
gbm_grid.best_estimator_.score(X_test, y_test)

In [ ]:
cm = pd.DataFrame(
    confusion_matrix(y_test, gbm_grid.best_estimator_.predict(X_test)),
    index=le.classes_,
    columns=le.classes_)
cm.style.background_gradient(cmap='Blues', axis=1)

### Know your metric
'prodliab' and 'medmal' are disproportionately classed as 'othliab'. This is a very common problem with imbalanced datasets. Underrepresented classes tend to perform poorly when optimizing for _accuracy_. In cases such as fraud detection accuracy would not be the best measure of model success.

How does one deal with imbalanced data?
- Some of the algorithms we explored `RandomForestClassifier` and `LGBMClassifier` have a `class_weight` hyperparameter that can be set to 'balanced'.
- Some algorithms support a `sample_weight` in the `fit` method. `model.fit(X, y, sample_weight=...)` this can also be used to give more weight to certain observations.
- Other packages like [imbalanced-learn](https://imbalanced-learn.org/stable/) are `sklearn` compatible packages that add additional functionality to upsample or downsample data to address class imbalance.

In all cases, you will likely use another metric (like Precision, Recall, F1, ROC) to gauge which model and hyperparameter set are the msot appropriate.


Consider an extremely imbalanced binary classification problem. Our featureset is made of random numbers that have no relationship to our response.

In [ ]:
X_imbalanced = np.random.rand(500, 20) # 20 columns of random uniform numbers
y_imbalanced = (np.random.rand(500)<.01)*1 # 1% of response variable is 1 99% is 0.

print('Dummy Accuracy on highly imbalanced data:',
      cross_validate(DummyClassifier(),
                     X_imbalanced,
                     y_imbalanced,
                     cv=cv)['test_score'].mean())

One simply needs to pass the `scoring` argument to `GridSeachCV` and/or `cross_validate` to chose a different scoring mechanism.

In [ ]:
rf_grid = GridSearchCV(
    RandomForestClassifier(),
    param_grid={'n_estimators': [100, 500]},
    scoring='f1_weighted', # change the scoring algorithm from accuracy to F1
    cv=cv)

rf_grid.fit(X_train, y_train)

In [ ]:
print('Default Random Forest F1:', cross_validate(RandomForestClassifier(), X_train, y_train, cv=cv, scoring='f1_weighted')['test_score'].mean())
print('Tuned Random Forest F1:', cross_validate(rf_grid.best_estimator_, X_train, y_train, cv=cv, scoring='f1_weighted')['test_score'].mean())

## Model Explainability

Model explainability is crucial for for several reasons. It aids the practitioner in identifying weaknesses in the model. It promotes refinement in feature engineering.  It promotes the discovery of _bias_ and _ethical_ concerns in the model. It's key for communicating what drives the model decisions and gaining stakeholder trust.


### Feature Importance

Feature importance is built into several algorithms. In particular the tree-based algorithms include this attribute and it shows the relative importance or contribution of each feature in a machine learning model's predictions.

For tree-based models, such as Decision Trees and Random Forests, feature importance is usually computed based on how much each feature contributes to reducing impurity or variance across the trees. The more a feature is used to split the data and the more it reduces the impurity, the higher its importance.

In GBMs, feature importance is determined by the average improvement in the model's loss function (e.g., mean squared error or log loss) that is achieved when a particular feature is used for splitting during the boosting process.

In [ ]:
pd.Series(
    rf_grid.best_estimator_.feature_importances_,
    index=rf_grid.best_estimator_.feature_names_in_
).sort_values(ascending=False).head(10)

In [ ]:
pd.Series(
    gbm_grid.best_estimator_.feature_importances_,
    index=gbm_grid.best_estimator_.feature_name_
).sort_values(ascending=False).head(10)

### Partial Dependence
Partial dependence of a feature provide a global view of the feature's impact, averaging across all instances in the dataset. In multiclassification, these plots show the probability of of the target class along a univariate view.

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

fig, ((ax0, ax1, ax2), (ax3, ax4, ax5)) = plt.subplots(ncols=3, nrows=2, figsize=(15,10))
features=[list(gbm_grid.best_estimator_.feature_name_).index('12_Paid_to_Incurred')]

PartialDependenceDisplay.from_estimator(gbm_grid.best_estimator_, X_train, features=features, target=[0], ax=ax0)
PartialDependenceDisplay.from_estimator(gbm_grid.best_estimator_, X_train, features=features, target=[1], ax=ax1)
PartialDependenceDisplay.from_estimator(gbm_grid.best_estimator_, X_train, features=features, target=[2], ax=ax2)
PartialDependenceDisplay.from_estimator(gbm_grid.best_estimator_, X_train, features=features, target=[3], ax=ax3)
PartialDependenceDisplay.from_estimator(gbm_grid.best_estimator_, X_train, features=features, target=[4], ax=ax4)
PartialDependenceDisplay.from_estimator(gbm_grid.best_estimator_, X_train, features=features, target=[5], ax=ax5);


Note that the probabilities shown are approximately in line with the balance between these classes.

In [ ]:
(pd.Series(le.inverse_transform(y_train)).value_counts() / len(y_train)).loc[le.classes_]

For further research into model explainability, other packages exist that go much deeper.

[`lime` (Local Interpretable Model-agnostic Explanations)](https://github.com/marcotcr/lime) and [`shap` (SHapley Additive exPlanations)](https://shap.readthedocs.io/en/latest/) are two popular techniques used for explaining the predictions of machine learning models. They both aim to provide local interpretability, meaning they explain individual predictions of a model rather than its overall behavior. Both LIME and SHAP are powerful tools for understanding the behavior of complex machine learning models and gaining insights into the factors that drive individual predictions. They provide interpretability to models without compromising their predictive power, helping practitioners, users, and stakeholders trust and use AI systems more effectively and responsibly.

## Concluding Tips

1. Prioritize Feature Engineering above all else.
2. Hyperparameter tuning is important but usually provides marginal improvements over feature engineering in traditional ML space.
3. GBMs are a nice blend of power and ease-of-use. They should be your goto for most tabular data problems.
4. Always cross-validate. Your training scores are lying to you.
5. The baseline performance you have to beat is not zero, its the NULL model.
6. Consider whether the default model performance metrics are appropriate for your specific task.
7. Use Pipelines to encapsulate multistep models. This reduces the risk of cross-contamination during cross-validation.

## Bonus - AutoML

Machine learning is fundamentally an exercise in experimentation.  AutoML gets rid of a lot of the tedium that goes into experimentation.

In [ ]:
from pycaret.classification import ClassificationExperiment

s = ClassificationExperiment()

AutoML generally involves setting up the base problem. For this example, we set up a multiclass problem which `pycaret` autodetects from our response variable. It also handles the train/test splitting.

In [ ]:
s.setup(
    data.reset_index().drop('GRNAME', axis=1),
    target='LOB',
    session_id=42)

Calling the `compare_models` method will compare several models available, including those we reviewed above. It further compares across various performance metrics.

In [ ]:
best = s.compare_models()

In [ ]:
print(best)

The best model of the bunch can further be tuned.

In [ ]:
tuned_model = s.tune_model(best)

In [ ]:
tuned_model